# Import Major Packages

In [313]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np

In [314]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

# Pose Visualization

In [315]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1730824342.542538    3452 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730824342.543349   23514 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-124-generic, LLVM 12.0.0)


In [316]:
results.face_landmarks.landmark[0].visibility

0.0

In [317]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [318]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

# Save into CSV File

In [321]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

# Capture to Train

In [324]:
class_name = "Sad"  # Optional: Add a class name for labeling


In [325]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1730824576.799046    3452 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730824576.799878   23778 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-124-generic, LLVM 12.0.0)


# Data Splitting

In [326]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [327]:
df = pd.read_csv('coords.csv')


In [328]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.462566,0.491452,-1.084257,0.999854,0.487987,0.437181,-1.024177,0.999685,0.505300,...,-0.005435,0.0,0.519036,0.428724,0.011075,0.0,0.523779,0.423726,0.011552,0.0
1,Happy,0.462078,0.492569,-1.060719,0.999846,0.487922,0.439001,-1.000241,0.999667,0.505010,...,-0.006114,0.0,0.519842,0.428242,0.010817,0.0,0.524565,0.423573,0.011293,0.0
2,Happy,0.462000,0.492737,-1.063127,0.999840,0.487946,0.439577,-1.002702,0.999657,0.504929,...,-0.005805,0.0,0.520575,0.425891,0.010900,0.0,0.525307,0.420843,0.011406,0.0
3,Happy,0.462136,0.492697,-1.063401,0.999838,0.488688,0.439621,-1.003636,0.999653,0.504996,...,-0.005505,0.0,0.521715,0.424175,0.011649,0.0,0.526365,0.419428,0.012187,0.0
4,Happy,0.462311,0.492645,-1.057700,0.999835,0.489200,0.439660,-0.995586,0.999644,0.505063,...,-0.005396,0.0,0.522982,0.424142,0.011979,0.0,0.527678,0.419281,0.012545,0.0


In [329]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
340,Sad,0.409682,0.761111,-0.922982,0.998807,0.430237,0.709744,-0.877774,0.997308,0.442852,...,-0.015999,0.0,0.472503,0.703585,-0.002131,0.0,0.475650,0.699685,-0.002200,0.0
341,Sad,0.419427,0.761582,-1.012545,0.998804,0.439758,0.710650,-0.967832,0.997335,0.452431,...,-0.016934,0.0,0.479908,0.709190,-0.002926,0.0,0.483009,0.705176,-0.002998,0.0
342,Sad,0.431403,0.762728,-1.030695,0.998733,0.450031,0.711853,-0.983951,0.997248,0.463129,...,-0.015998,0.0,0.489571,0.716004,-0.001177,0.0,0.492713,0.712152,-0.001154,0.0
343,Sad,0.451852,0.776382,-1.114889,0.997758,0.467088,0.723014,-1.074398,0.995705,0.479701,...,-0.016089,0.0,0.500194,0.723495,-0.000622,0.0,0.503416,0.719499,-0.000559,0.0
344,Sad,0.465064,0.791987,-1.129894,0.997585,0.479933,0.738749,-1.096957,0.995340,0.491980,...,-0.017469,0.0,0.514773,0.730197,-0.001688,0.0,0.518111,0.725764,-0.001658,0.0


In [330]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [331]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [332]:
X_train.shape

(241, 2004)

In [333]:
X_test.shape

(104, 2004)

In [334]:
y_test.shape


(104,)

# Train

In [335]:
'''The first two are used to make the pipeline it self
'''
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
'''Used to Train the model using one of the models , and used 
for hyperparameters tuning'''
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

In [336]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [337]:
for name, pipeline in pipelines.items():
    print(f"Pipeline Key Name '{name}':")
    print(pipeline)
    print("-" * 45)  # Divider for clarity


Pipeline Key Name 'lr':
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])
---------------------------------------------
Pipeline Key Name 'rc':
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridgeclassifier', RidgeClassifier())])
---------------------------------------------
Pipeline Key Name 'rf':
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])
---------------------------------------------
Pipeline Key Name 'gb':
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])
---------------------------------------------


In [338]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [339]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Happy',
       'Happy'], dtype=object)

# Model Evaluation

In [340]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [341]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [342]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy',
       'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Sad',
       'Happy', 'Sad', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy',
       'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad', 'Sad', 'Happy',
       'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad',
       'Sad', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Sad', 'Sad', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy',
       'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Happy', 'Sad',
       'Happy', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy', 'Sad', 'Happy',
       'Happy'], dtype=object)

In [343]:
y_test

202      Sad
286      Sad
305      Sad
66     Happy
37     Happy
       ...  
221      Sad
166    Happy
277      Sad
93     Happy
52     Happy
Name: class, Length: 104, dtype: object

In [344]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Test and Detection

In [345]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [346]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [347]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1730825863.793772    3452 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730825863.794586   25323 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: AMD RENOIR (DRM 3.42.0, 5.15.0-124-generic, LLVM 12.0.0)
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler w

Sad [0.2 0.8]
Sad [0.2 0.8]


/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.2 0.8]
Sad [0.2 0.8]
Sad [0.2 0.8]


/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Happy [1. 0.]
Happy [1. 0.]
Happy [1. 0.]


/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [1. 0.]
Happy [1. 0.]
Happy [1. 0.]


/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/omar_ben_emad/.local/lib/python3.8

Happy [1. 0.]
Happy [1. 0.]


In [309]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(289, 246)